# Getting New Data

In [14]:
#For testing, import the user name
from env import api_key
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key
from time import sleep
import prepare

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

### Combine the Code Above Into One Function

In [15]:
def get_new_patch_data(name_list, api_key):
    """
    This function takes in a list of usernames that are in the same tier and iterates 
    through them to find their puuids.
    It will then use those puuids to grab all of their match ids that are in the newest patch.
    From there, the function will iterate through all of the match ids and grab the timeline
    data for each match as well as the other game data. 
    Finally, this function will incrementally save the returned json information every 10 times.
    This prevents the loss of all data if the function errors out at some point.
    This function returns nothing.
    """
    
    #Set up the basegame number. Any match_ids below this number are part of the old patch
    #and will not be included. We will use the basegame var for comparisons in the loop
    basegame = 4107000001
    
    #Set up the base url for grabbing match_ids
    match_url = 'https://americas.api.riotgames.com/'
    
    #Set up the base url for grabbing puuids
    puuid_base_url = 'https://na1.api.riotgames.com/'
    
    #Set up another counter to keep track of how many users have been completed
    n = 0
    
    #Set up a counter to keep track of how many json files have been completed in total
    num_json = 0
    
    #Set up one more counter to keep track of how many pulls we have made.
    #Can't go over 100 pulls every 2 minutes.
    num_pulls = 0
    
    #Set up empty lists to store timeline and other game data
    timeline_data = []
    game_data = []
    
    #Loop through each name in the given name_list
    #First grab the puuid, then grab the match_ids, then actual match json info
    for name in name_list:
        
        #for testing
        print(f'Beginning Loop for user {name}')
        
        #Update n var
        n += 1
        
        #Leave a status update after every 100 users
        if n % 100 == 0:
            print(f'Completed {n} users of {len(name_list)}!\n')
        
        ########################## Get puuids ##########################
        
        #Set up the query
        query = f'lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
        
        #Get the response
        response = requests.get(puuid_base_url + query)
        
        #Update pull count and sleep for 150 seconds if necessary
        num_pulls += 1
        
        if num_pulls % 100 == 0:
            print('\nHit 100 pulls! Sleeping for 150 seconds.')
            sleep(150)
            print('Continuing!\n')
        
        #Check the response code and leave message if something goes wrong
        if response.status_code != 200:
            print(f'Something went wrong! Status Code: {response.status_code}. Did not get puuid for user: {name}')
            print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
            continue
        
        #Grab the user's puuid
        puuid = response.json()['puuid']
        
        #For testing
        print(f'\t-Got the puuid for user {name}.')
        
        ########################## Get Match IDs #########################
        
        #Now that we have a single puuid, grab all the match ids for that user that are greater
        #than the basegame value. These match_ids will be for the newest patch.
        
        #Set up a counter
        i = 0
        
        #This loop will be set up to run indefinitely. Be sure to break out of it when 
        #Done with the current user
        while i >= 0:
            #Set up the query to get a match_id
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i}&count=1&api_key={api_key}'
            
            #Get the response
            response = requests.get(match_url + query)
            
            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response status code and leave message if something went wrong
            if response.status_code != 200:
                print(f'Something went wrong! Status Code: {response.status_code}. Did not get Match_ID for user: {name}')
                print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
                break
                
            #Grab the match_id
            match = response.json()
            
            #Verify there's something in it
            if len(match) == 0:
                print(f'\t-No match ID for user {name}. Skipping this user!')
                break
            
            #Convert match_id to integer
            match_num = int(match[0].strip('NA1_'))
            
            #Compare current match_id to basegame var.
            #Skip if match_id is less than basegame
            if match_num < basegame:
                print(f'\t-Completed getting all current patch Match IDs for user {name}')
                print(f'\t-There are {len(name_list) - n} users left.')
                break

            ####################### Get Match Info ##########################
            
            #Now that we know the current match_id is part of the newest patch,
            #We can retrieve the actual match info. Keep in mind this is still part
            #of the while loop above
                
            #Every 250 entries, save the lists as json files and reset the lists
            if len(timeline_data) % 500 == 0:    
                #Save timeline data.
                #First, convert it to a df
                timeline_df = pd.DataFrame(timeline_data)

                #Save as json file. Use username string to identify whose file it is
                timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created timeline_data_user_{name}_end_{num_json}.json')

                #Save other game data
                #First, convert it to a df
                game_df = pd.DataFrame(game_data)

                #Save as json file. Use username string to identify whose file it is
                game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created other_game_data_user_{name}_end_{num_json}.json\n')
                
                #Now reset the temp lists
                timeline_data = []
                game_data = []
            
        
            ############################# TIMELINE DATA #######################
            #Set up timeline url
            timeline_query = f'lol/match/v5/matches/{match[0]}/timeline/?api_key={api_key}'

            #Grab timeline json data
            timeline_response = requests.get(match_url + timeline_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message
            if timeline_response.status_code != 200:
                print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID.\n')
                continue

            #Turn it into json format
            timeline_json = timeline_response.json()

            #Append this data to the timeline_data list
            timeline_data.append(timeline_json)
            
            ############################# OTHER GAME DATA #######################
            #Set up game data url
            game_query = f'lol/match/v5/matches/{match[0]}?api_key={api_key}'

            #Grab game json data
            game_response = requests.get(match_url + game_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message.
            if game_response.status_code != 200:
                print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')

                #Remove the last entry in the timeline_data list
                timeline_data.pop()
                continue

            #Turn it into json format
            game_json = game_response.json()

            #Append this data to the game_data list
            game_data.append(game_json)
            
            #Update num_json
            num_json += 1
            
            #Update i
            i += 1
            
        #Tell the user how many match_ids were used for the current user
        print(f'\t-Completed {i} Match IDs for user {name}\n')
    
    #Save any json data that was not saved at the end of the for loop
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)

    #Save as json file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created timeline_data_user_{name}_end_{num_json}.json')

    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)

    #Save as json file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created other_game_data_user_{name}_end_{num_json}.json')
    
    #Complete! Leave a message
    print('All users completed! Follow suit.')
            

# Get the New Data

Now that the function has been completed, get the desired data. Joshua C. is gathering the data for the pro players. I will get the data using the names from Jared's list.

In [16]:
#Load the names csv
name_list = pd.read_csv('2640_players.csv', index_col = [0])
name_list = name_list[1000:1500]

In [17]:
name_list = list(name_list.players)

In [ ]:
#Run the function
get_new_patch_data(name_list, api_key)

Beginning Loop for user Emilia
	-Got the puuid for user Emilia.
	-Created timeline_data_user_Emilia_end_0.json
	-Created other_game_data_user_Emilia_end_0.json

	-Completed getting all current patch Match IDs for user Emilia
	-There are 499 users left.
	-Completed 10 Match IDs for user Emilia

Beginning Loop for user davidfighting
	-Got the puuid for user davidfighting.
	-Completed getting all current patch Match IDs for user davidfighting
	-There are 498 users left.
	-Completed 5 Match IDs for user davidfighting

Beginning Loop for user Viet Conq Heimer
	-Got the puuid for user Viet Conq Heimer.
	-Completed getting all current patch Match IDs for user Viet Conq Heimer
	-There are 497 users left.
	-Completed 1 Match IDs for user Viet Conq Heimer

Beginning Loop for user Lagopotamus
	-Got the puuid for user Lagopotamus.
	-Completed getting all current patch Match IDs for user Lagopotamus
	-There are 496 users left.
	-Completed 3 Match IDs for user Lagopotamus

Beginning Loop for user I 

	-Got the puuid for user SeckSi Bacon.
	-Completed getting all current patch Match IDs for user SeckSi Bacon
	-There are 462 users left.
	-Completed 2 Match IDs for user SeckSi Bacon

Beginning Loop for user always plan ahea
	-Got the puuid for user always plan ahea.
	-Completed getting all current patch Match IDs for user always plan ahea
	-There are 461 users left.
	-Completed 7 Match IDs for user always plan ahea

Beginning Loop for user pixxxx
	-Got the puuid for user pixxxx.
	-Completed getting all current patch Match IDs for user pixxxx
	-There are 460 users left.
	-Completed 1 Match IDs for user pixxxx

Beginning Loop for user insung
	-Got the puuid for user insung.
	-Completed getting all current patch Match IDs for user insung
	-There are 459 users left.
	-Completed 0 Match IDs for user insung

Beginning Loop for user Korena
	-Got the puuid for user Korena.
	-Completed getting all current patch Match IDs for user Korena
	-There are 458 users left.
	-Completed 2 Match IDs for u